In [4]:
import os

In [5]:
# Set generic variables
%run set_env.ipynb

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark Session with Default Configurations").getOrCreate()

### Set path to CSV file in the Iguazio Platform

In [ ]:
file_path = os.path.join(os.getenv('V3IO_HOME_URL')+'/%s'% csv_file)
file_path

## Load CSV file into a Spark Dataframe

In [7]:
%%time
df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(file_path)

CPU times: user 63.7 ms, sys: 16.3 ms, total: 80.1 ms
Wall time: 9min 8s


### Explore Dataframe

In [10]:
%%time
df.head(5)

CPU times: user 5.59 ms, sys: 0 ns, total: 5.59 ms
Wall time: 259 ms


[Row(usaf='999999', wban=4237, datetime=datetime.datetime(2020, 6, 13, 4, 0), latitude=47.513999999999996, longitude=-123.81200000000001, elevation=87.0, windAngle=0.0, windSpeed=0.0, temperature=11.7, seaLvlPressure=None, cloudCoverage=None, presentWeatherIndicator=None, pastWeatherIndicator=None, precipTime=1.0, precipDepth=2.0, snowDepth=None, stationName='QUINAULT 4 NE', countryOrRegion='US', p_k='999999-04237', year=2020, day=13, version=1.0),
 Row(usaf='999999', wban=4990, datetime=datetime.datetime(2020, 6, 30, 7, 20), latitude=43.735, longitude=-96.62200000000001, elevation=486.0, windAngle=None, windSpeed=None, temperature=24.0, seaLvlPressure=None, cloudCoverage=None, presentWeatherIndicator=None, pastWeatherIndicator=None, precipTime=None, precipDepth=None, snowDepth=None, stationName='SIOUX FALLS 14 NNE', countryOrRegion='US', p_k='999999-04990', year=2020, day=30, version=1.0),
 Row(usaf='999999', wban=4223, datetime=datetime.datetime(2020, 6, 17, 1, 50), latitude=48.54100

In [11]:
%%time
df.describe()

CPU times: user 88.3 ms, sys: 9.27 ms, total: 97.6 ms
Wall time: 11min 38s


DataFrame[summary: string, usaf: string, wban: string, latitude: string, longitude: string, elevation: string, windAngle: string, windSpeed: string, temperature: string, seaLvlPressure: string, cloudCoverage: string, presentWeatherIndicator: string, pastWeatherIndicator: string, precipTime: string, precipDepth: string, snowDepth: string, stationName: string, countryOrRegion: string, p_k: string, year: string, day: string, version: string]

In [12]:
%%time
df.count()

CPU times: user 5.98 ms, sys: 470 µs, total: 6.45 ms
Wall time: 36.1 s


146451216

### Prep Dataframe for SparkSQL 

In [9]:
%%time
df.createOrReplaceTempView('weather')

CPU times: user 1.5 ms, sys: 224 µs, total: 1.73 ms
Wall time: 24.4 ms


In [14]:
%%time
sqlDF = spark.sql("SELECT * FROM weather limit 10")

CPU times: user 0 ns, sys: 2.06 ms, total: 2.06 ms
Wall time: 99 ms


In [10]:
%%time
sqlDF = spark.sql("SELECT wban,count(*) FROM weather group by wban")

CPU times: user 782 µs, sys: 1.03 ms, total: 1.81 ms
Wall time: 28.2 ms


In [11]:
%%time
sqlDF.show()

+-----+--------+
| wban|count(1)|
+-----+--------+
|  471|   29633|
|34061|   27370|
|94950|   13127|
|26623|   13533|
|26425|   15007|
| 3997|   13363|
| 3749|     732|
|23271|     745|
| 4935|   29527|
|22521|   12568|
|  463|   29459|
|  148|   29161|
|13840|   13735|
| 3918|   13467|
| 3179|   12358|
|93037|   11664|
|54844|   38216|
|  243|   25490|
|23136|   12324|
|  392|   45309|
+-----+--------+
only showing top 20 rows

CPU times: user 7.02 ms, sys: 3.95 ms, total: 11 ms
Wall time: 1min 5s


### Write Dataframe to Parquet

In [16]:
%%time
df.write.parquet("v3io://users/admin/data/weather_parquet_spark_snappy/", mode='overwrite',compression='snappy')

CPU times: user 51.6 ms, sys: 10.8 ms, total: 62.4 ms
Wall time: 7min 9s


In [17]:
%%time
df.write.parquet("v3io://users/admin/data/weather_parquet_spark_no_compress/", mode='overwrite')

CPU times: user 49.2 ms, sys: 11.6 ms, total: 60.7 ms
Wall time: 7min


In [18]:
spark.stop()